In [19]:
from __future__ import print_function, division

import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, utils
import time
import os
import copy
import torch.nn as nn
import torch.nn.functional as F
import datetime

# TODO: Implement a convolutional neural network (https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html)
class Net(nn.Module):
    """
    Input - 1x32x32
    Output - 10
    """
    def __init__(self):
        super(Net, self).__init__()
        
        self.params = {'conv':[(), 
                               (3, 16, 5, 1, 1), 
                               (16, 32, 3, 1, 1),
                               (32, 32, 3, 1, 0),
                               (32, 16, 3, 1, 1)], # in_channels, out_channels, kernel_size, stride, padding
                       'pool':[(), 
                               (2, 2, 0),
                               (2, 2, 0)], # kernel_size, stride, padding
                       'fc':[(), 
                             (16*6*6, 120),
                             (120, 90), 
                             (90, 10)], # in_channels, out_channels
                       'drop':[0, 
                               0.25, 
                               0.25]
                      }
        
        self.conv1 = nn.Conv2d(*self.params['conv'][1])
        self.conv2 = nn.Conv2d(*self.params['conv'][2])
        self.conv3 = nn.Conv2d(*self.params['conv'][3])
        self.conv4 = nn.Conv2d(*self.params['conv'][4])
        
        self.pool1 = nn.MaxPool2d(*self.params['pool'][1])
        self.pool2 = nn.MaxPool2d(*self.params['pool'][2])
        
        self.fc1 = nn.Linear(*self.params['fc'][1])
        self.fc2 = nn.Linear(*self.params['fc'][2])
        self.fc3 = nn.Linear(*self.params['fc'][3])
        
        # self.drop1 = nn.Dropout2d(self.params['drop'][1])
        # self.drop2 = nn.Dropout2d(self.params['drop'][2])
        
        self.printed = False

        # TODO: Initialize layers

    def forward(self, img):

        # TODO: Implement forward pass
        x = img
        
        x = F.relu(self.conv1(x))
        if not self.printed: 
            print("CONV1", x.size())
        x = F.relu(self.conv2(x))
        if not self.printed: 
            print("CONV2", x.size())
        x = self.pool1(x)
        if not self.printed: 
            print("POOL1", x.size())
        
        x = F.relu(self.conv3(x))
        if not self.printed: 
            print("CONV3", x.size())
        x = F.relu(self.conv4(x))
        if not self.printed: 
            print("CONV4", x.size())
        x = self.pool2(x)
        if not self.printed: 
            print("POOL2", x.size())
        
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        if not self.printed: 
            print("FC1", x.size())
        '''x = self.drop1(x)
        if not self.printed: 
            print("DROP1", x.size())'''
        x = F.relu(self.fc2(x))
        if not self.printed: 
            print("FC2", x.size())
        x = self.fc3(x)
        if not self.printed: 
            print("FC3", x.size())
            self.printed = True

        return x

In [20]:
def test(model):
    
    # testing overall correct rate
    correct = 0
    total = 0
    
    with torch.no_grad():
        for i, data in enumerate(dataloaders['train'], 0):
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Training accuracy: %d %%' % (100 * correct / total))
    
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Testing accuracy: %d %%' % (100 * correct / total))
    
    # prepare to count predictions for each class
    correct_pred = {classname: 0 for classname in class_names}
    total_pred = {classname: 0 for classname in class_names}

    # again no gradients needed
    with torch.no_grad():
        for i, data in enumerate(dataloaders['test'], 0):
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[class_names[label]] += 1
                total_pred[class_names[label]] += 1


    # print accuracy for each class
    print("Testing accuracy (each class): ")
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print("{:1s}: {:.1f}%;  ".format(classname, accuracy), end=' ')
    print()

In [21]:
def train_test(model, criterion, optimizer, scheduler, num_epochs=25):
    
    for epoch in range(num_epochs):  

        running_loss = 0.0
        loss_record=[]
        for i, data in enumerate(dataloaders['train'], 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                loss_record.append(running_loss / 2000)
                # print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                
        print(datetime.datetime.now(), ' Epoch', (epoch + 1), ': Average Loss', loss_record)

    print('Finished Training')
    
    test(model)
    
    return None

In [22]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((32,32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset initialization
data_dir = 'data' # Suppose the dataset is stored under this folder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']} # Read train and test sets, respectively.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0) for x in ['train', 'test']}
# trainloader = torch.utils.data.DataLoader(image_datasets['train'], batch_size=4, shuffle=True, num_workers=2)
# teatloader = torch.utils.data.DataLoader(image_datasets['test'], batch_size=4, shuffle=True, num_workers=2)

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # Set device to "cpu" if you have no gpu


In [27]:
model_ft = Net() # Model initialization

model_ft = model_ft.to(device) # Move model to cpu

criterion = nn.CrossEntropyLoss() # Loss function initialization

# TODO: Adjust the following hyper-parameters: learning rate, decay strategy, number of training epochs.
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-4) # Optimizer initialization

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=20, gamma=0.1) # Learning rate decay strategy

epoch = 10

for n in range((epoch // 5)):
    print(datetime.datetime.now())
    epo = 5*n+5
    print("epoch range: ", epo-4, " to ", epo)
    train_test(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

PATH = './cnn_a2_.pth'
torch.save({
        'epoch': epoch,
        'model_state_dict': model_ft.state_dict(),
        'optimizer_state_dict': optimizer_ft.state_dict()
        }, PATH)

2021-05-07 23:13:53.383946
epoch range:  1  to  5
CONV1 torch.Size([4, 16, 30, 30])
CONV2 torch.Size([4, 32, 30, 30])
POOL1 torch.Size([4, 32, 15, 15])
CONV3 torch.Size([4, 32, 13, 13])
CONV4 torch.Size([4, 16, 13, 13])
POOL2 torch.Size([4, 16, 6, 6])
FC1 torch.Size([4, 120])
FC2 torch.Size([4, 90])
FC3 torch.Size([4, 10])
2021-05-07 23:14:49.997404  Epoch 1 : Average Loss [2.1803382738232613, 1.4896714662909507, 1.3466099384836852]
2021-05-07 23:15:44.674484  Epoch 2 : Average Loss [1.2107083199378104, 1.1377596237827092, 1.0602725045438857]
2021-05-07 23:16:46.389851  Epoch 3 : Average Loss [0.9592864093724638, 0.9086817204570398, 0.8697101417221129]
2021-05-07 23:17:45.888094  Epoch 4 : Average Loss [0.7934361056776251, 0.7750622836628464, 0.735636769162491]
2021-05-07 23:18:45.138482  Epoch 5 : Average Loss [0.7038499693279154, 0.681078308247932, 0.6517290979779791]
Finished Training
Training accuracy: 80 %
Testing accuracy: 78 %
Testing accuracy (each class): 
0: 83.4%;   1: 79.0%

NameError: name 'loss' is not defined

In [26]:
model = Net()
optimizer = optim.Adam(model_ft.parameters(), lr=1e-4)
PATH = './cnn_a2_.pth'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

test(model)

FileNotFoundError: [Errno 2] No such file or directory: './cnn_a2_.pth'